In [1]:
# Travel time cell
from datetime import datetime, timedelta
from pytz import timezone as ptz

DATE_FORMAT = "%Y-%m-%dT%H:%M:%-SZ"
oneDay = timedelta(1)

departureTimeOptions = {
    'now': lambda tz: datetime.now(tz).replace(minute=59, second=59).isoformat(),
    'morning': lambda tz: (datetime.now(tz).replace(hour=9, minute=0, second=0) + oneDay).isoformat(),
    'noon': lambda tz: (datetime.now(tz).replace(hour=12, minute=0, second=0) + oneDay).isoformat(),
    'afternoon': lambda tz: (datetime.now(tz).replace(hour=5, minute=0, second=0) + oneDay).isoformat(),
    'evening': lambda tz: (datetime.now(tz).replace(hour=7, minute=0, second=0) + oneDay).isoformat(),
}

print("now time: " + departureTimeOptions['now'](ptz('EST')))
print("morning time: " + departureTimeOptions['morning'](ptz('EST')))
print(datetime.now().timestamp())

now time: 2023-10-01T20:59:59.251279-05:00
morning time: 2023-10-02T09:00:00.251474-05:00
1696211875.25154


In [6]:
import math
import requests
from const import GOOGLE_MAPS_API_KEY
import json


COMPUTE_ROUTE_URL = 'https://routes.googleapis.com/directions/v2:computeRoutes'
MY_HOUSE = '3025 The Credit Woodlands, Mississauga, ON L5C 2V3'
SQUARE_ONE_ADDRESS = '100 City Centre Dr, Mississauga, ON L5B 2C9'


def getDurationSec(route): return int(route["duration"][0:-1])


def findFastestRoute(response) -> timedelta:
    routeDurations = json.loads(response.text)["routes"]
    fastestCommute = math.inf

    for r in routeDurations:
        fastestCommute = min(getDurationSec(r), fastestCommute)

    return timedelta(seconds=fastestCommute)


DISTANCE_AND_LINE_FIELDMASK_HEADER = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': GOOGLE_MAPS_API_KEY,
    'X-Goog-FieldMask': 'routes.duration,routes.distanceMeters,routes.polyline.encodedPolyline',
}

DURATION_FIELDMASK_HEADER = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': GOOGLE_MAPS_API_KEY,
    'X-Goog-FieldMask': 'routes.duration',
}

TRANSIT_DETAIL_FIELDMASK_HEADER = {
        'content-type': 'application/json; application/json',
        'X-Goog-Api-Key': GOOGLE_MAPS_API_KEY,
        'X-Goog-FieldMask': 'routes.legs.steps.transitDetails',
    }

def driveRouteParams(originAddress: str, destinationAddress: str, departureOption='now')-> dict:
    return {
        'origin': {
            'address': originAddress
        },
        'destination': {
            'address': destinationAddress
        },
        'travelMode': 'DRIVE',
        'routingPreference': 'TRAFFIC_AWARE',
        'departureTime': departureTimeOptions[departureOption](ptz('EST')),
        'computeAlternativeRoutes': False,
        'routeModifiers': {
            'avoidTolls': True,
            'avoidHighways': False,
            'avoidFerries': True,
        },
    }

def transitRouteParam(originAddress: str, destinationAddress: str, departureOption='now'):
    return {
        'origin': {
            'address': originAddress
        },
        'destination': {
            'address': destinationAddress
        },
        'travelMode': 'TRANSIT',
        'computeAlternativeRoutes': True,
        'transitPreferences': {
            'routingPreference': 'FEWER_TRANSFERS',
        },
        'departureTime': departureTimeOptions[departureOption](ptz('EST')),
    }

def bicycleRouteParam(originAddress: str, destinationAddress: str, departureOption='now'):
    return {
        'origin': {
            'address': originAddress
        },
        'destination': {
            'address': destinationAddress
        },
        'travelMode': 'BICYCLE',
        'departureTime': departureTimeOptions[departureOption](ptz('EST')),
    }

def computeRoute(fieldMask, travelMode, orginAddr, destAddr, departureOption='now'):
    travelParam = {
        'DRIVE': driveRouteParams,
        'TRANSIT': transitRouteParam,
        'BICYCLE': bicycleRouteParam,
    }
    return requests.post(COMPUTE_ROUTE_URL, headers=fieldMask, json=travelParam[travelMode](orginAddr, destAddr, departureOption) )

def commuteTime(travelMode, orginAddr, destAddr, departureOption='now'):
    return findFastestRoute(computeRoute(DURATION_FIELDMASK_HEADER, travelMode, orginAddr, destAddr, departureOption))


print(computeRoute(DISTANCE_AND_LINE_FIELDMASK_HEADER, 'DRIVE', MY_HOUSE, SQUARE_ONE_ADDRESS).text)

print(f'travel times from My_House: ${MY_HOUSE} to Square One ${SQUARE_ONE_ADDRESS}')
print("Fastest Driving Commute: ")
print(commuteTime('DRIVE', MY_HOUSE, SQUARE_ONE_ADDRESS))

print("Fastest transit commute: ")
print(commuteTime('TRANSIT', MY_HOUSE, SQUARE_ONE_ADDRESS))


print("Fastest Bicycle Commute:")
print(commuteTime('BICYCLE', MY_HOUSE, SQUARE_ONE_ADDRESS))



{
  "routes": [
    {
      "distanceMeters": 6812,
      "duration": "674s",
      "polyline": {
        "encodedPolyline": "omyhGtlceNkCuD?Gf@{@LCXN|EtFXd@DRCXq@tAb@l@tA{EF]u@g@uAw@cB}AqBaCSIK?qCeDsCcDiDcE{@}@gFoGoBmC}GcIkEkFqCaDkCkDqJiL}@eAc@cAiJuKeDzEg@~@Of@qEnHyG|K_@Tw@fAkEbH}GzK]b@Kd@m@fA_KbPcC~DoCjEwF`JmAtAsAvAUHc@f@oDdFyCuDeBoBaGqGiLaNiFoGyCkDoRsU{LqOgIpOGRURGDgAD"
      }
    }
  ]
}

travel times from My_House: $3025 The Credit Woodlands, Mississauga, ON L5C 2V3 to Square One $100 City Centre Dr, Mississauga, ON L5B 2C9
Fastest Driving Commute: 
0:11:14
Fastest transit commute: 
0:26:26
Fastest Bicycle Commute:
0:27:16


In [21]:
import pandas as pd

df = pd.read_csv("Kitchener Waterloo Housing.csv")

def getListOfAddresses(df):  
  return df['Address'] + ', '  +df['City'] + ', ' + df['Province'] + ' ' +  df['Postal Code']

addressList = getListOfAddresses(df)

# print(addressList)

for address in addressList:
  print(address)


commute times to my house
98 Foxglove Cres, Kitchener, ON N2E 3Y8
228 Bankside Drive, Kitchener, ON N2N 3E5
240 Westmeadow Drive, Kitchener, ON N2N 0A1
